In [4]:
import numpy as np
import pandas as pd
from sklearn import *

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,22256635,"Nonsense? kiss off, geek. what I said is true...",1,0,0,0,0,0
1,27450690,"""\n\n Please do not vandalize pages, as you di...",0,0,0,0,0,0
2,54037174,"""\n\n """"Points of interest"""" \n\nI removed the...",0,0,0,0,0,0
3,77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0
4,79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0


In [7]:
y_cols = [c for c in train.columns if c not in ['id','comment_text']]
y = train[y_cols]
test_ids = test['id'].values

In [8]:
df = pd.concat([train['comment_text'], test['comment_text']], axis=0)
df = df.fillna("test")
n_row = train.shape[0]

Векторизуем нашу модель в tfidf матрицу

Причем возьмем слова и из теста

In [26]:
tfidf = feature_extraction.text.TfidfVectorizer(min_df=3, max_df=0.9, strip_accents='unicode',
                                                analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,3), 
                                                use_idf=1, stop_words='english')
data = tfidf.fit_transform(df)

Обучим две модели

Начнем с extratrees

In [26]:
model = ensemble.ExtraTreesClassifier(n_jobs=-1, n_estimators=100)
model = model.fit(data[:nrow], y)

1


In [29]:
sub1 = pd.DataFrame(model.predict(data[nrow:]))
sub1.columns = y_cols
sub1['id'] = test_ids

1


In [30]:
sub1.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,6044863,0.001864,1.125066e-06,0.000339,1.315842e-05,0.000112,0.000093
1,6102620,0.000204,2.440083e-08,0.000046,3.298898e-07,0.000007,0.000006
2,14563293,0.000186,3.065889e-08,0.000046,3.746936e-07,0.000007,0.000006
3,21086297,0.005284,3.060414e-06,0.000711,3.353372e-05,0.000326,0.000214
4,22982444,0.006287,5.782117e-06,0.000936,6.262553e-05,0.000469,0.000328


Теперь логрегрессия

In [50]:
model = linear_model.LogisticRegression(C=0.5, penalty='l1')
model = model.fit(data[:nrow], y)

1


In [51]:
sub2 = pd.DataFrame(model.predict(data[nrow:]))
sub2.columns = y_cols
sub2['id'] = test_ids

1


In [52]:
sub2.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,6044863,0.004927,1.075040e-05,0.000681,1.981909e-05,0.000526,0.000110
1,6102620,0.000213,1.973792e-07,0.000056,2.027216e-07,0.000015,0.000003
2,14563293,0.000159,1.054693e-07,0.000039,1.154329e-07,0.000010,0.000001
3,21086297,0.015406,3.838584e-05,0.002095,7.668863e-05,0.002036,0.000347
4,22982444,0.002901,5.016423e-06,0.000465,9.698233e-06,0.000302,0.000060


Блендинг

In [73]:
blend = pd.merge(sub1, sub2, how='left', on='id')
for c in y_cols:
    if c != 'id':
        blend[c] = blend[c+'_x'] * 0.5 + blend[c+'_y'] * 0.5
        blend[c] = blend[c].clip(0+1e12, 1-1e12)
blend = balend[sub1.columns]
blend.to_csv('blend.csv', index=False)

1


In [74]:
blend.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,6044863,0.003396,5.937734e-06,0.000510,1.648875e-05,0.000319,0.000102
1,6102620,0.000208,1.108900e-07,0.000051,2.663057e-07,0.000011,0.000004
2,14563293,0.000173,6.806408e-08,0.000042,2.450633e-07,0.000008,0.000004
3,21086297,0.010345,2.072312e-05,0.001403,5.511117e-05,0.001181,0.000280
4,22982444,0.004594,5.399270e-06,0.000700,3.616188e-05,0.000385,0.000194
